In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
from features.feature_engineering import build_feature_pipeline
from sklearn.model_selection import train_test_split
from features.encoder import ManualWOEEncoder as WOEEncoderWrapper

import pandas as pd

In [20]:
df=pd.read_parquet('../../data/interim/clean_data.parquet')

In [21]:
df.sample(5)

,AccountId,CustomerId,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_weekday,is_large_transaction,Amount_log,Amount_capped,is_outlier_Amount
53424,AccountId_4249,CustomerId_7343,ProviderId_4,ProductId_10,airtime,ChannelId_2,-30000.0,30000,2019-01-08 10:53:57+00:00,4,0,10,8,1,1,False,10.308986,-30000.0,0
31895,AccountId_571,CustomerId_908,ProviderId_6,ProductId_3,airtime,ChannelId_3,2000.0,2000,2018-12-18 15:43:44+00:00,2,0,15,18,12,1,False,7.601402,2000.0,0
93156,AccountId_4841,CustomerId_682,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-11 06:51:58+00:00,2,0,6,11,2,0,False,3.931826,-50.0,0
19709,AccountId_2237,CustomerId_2649,ProviderId_3,ProductId_21,utility_bill,ChannelId_3,10000.0,11200,2018-12-07 14:49:28+00:00,1,0,14,7,12,4,False,9.210440,10000.0,0
30549,AccountId_4421,CustomerId_4878,ProviderId_1,ProductId_15,financial_services,ChannelId_3,100000.0,100000,2018-12-17 11:45:46+00:00,2,0,11,17,12,0,True,11.512935,80000.0,1


In [22]:
df.columns

Index(['AccountId', 'CustomerId', 'ProviderId', 'ProductId', 'ProductCategory',
       'ChannelId', 'Amount', 'Value', 'TransactionStartTime',
       'PricingStrategy', 'FraudResult', 'transaction_hour', 'transaction_day',
       'transaction_month', 'transaction_weekday', 'is_large_transaction',
       'Amount_log', 'Amount_capped', 'is_outlier_Amount'],
      dtype='object')

In [23]:
X = df
y = df['FraudResult']

X_train, X_val, y_train, y_val = train_test_split(X, X['FraudResult'], test_size=0.2, random_state=42)


In [24]:
pipeline = build_feature_pipeline()
pipeline.fit(X_train, y_train)
X_train_transformed = pipeline.transform(X_train)
X_val_transformed = pipeline.transform(X_val)                   

In [25]:
print(X_train_transformed.shape)
print(X_val_transformed.shape)
print(type(X_train_transformed))

(76529, 23)
(19133, 23)
<class 'numpy.ndarray'>


In [26]:
from sklearn.compose import ColumnTransformer

preprocessor = pipeline.named_steps['preprocessor']

# For sklearn >=1.0
feature_names = preprocessor.get_feature_names_out()

print(feature_names)

['datetime__TransactionStartTime_hour'
 'datetime__TransactionStartTime_dayofweek' 'numeric__Amount'
 'numeric__Value' 'numeric__Amount_log' 'numeric__Amount_capped'
 'numeric__total_transaction_amount' 'numeric__avg_transaction_amount'
 'numeric__transaction_count' 'numeric__std_transaction_amount'
 'woe__ProductCategory_woe' 'woe__ChannelId_woe' 'woe__ProviderId_woe'
 'woe__ProductId_woe' 'onehot__PricingStrategy_0'
 'onehot__PricingStrategy_1' 'onehot__PricingStrategy_2'
 'onehot__PricingStrategy_4' 'onehot__is_large_transaction_0'
 'onehot__is_large_transaction_1']
